In [1]:
!pip install -U trl datasets peft flash-attn transformers bitsandbytes accelerate


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig #TrainingArguments
from peft import LoraConfig, get_peft_model
import torch

In [3]:
model_id = "Qwen/Qwen2.5-0.5B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    #attn_implementation="flash_attention_2"
)

In [4]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj"
    ],
    task_type="CAUSAL_LM",
)

In [5]:
lora_model = get_peft_model(model, peft_config)
lora_model.print_trainable_parameters()

trainable params: 8,798,208 || all params: 502,830,976 || trainable%: 1.7497


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [7]:
print(tokenizer.eos_token)

<|im_end|>


In [8]:
dataset = load_dataset("bbz662bbz/databricks-dolly-15k-ja-gozarinnemon", split="train")
dataset = dataset.filter(lambda example: example["input"] == "")

In [9]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### 質問:\n{example['instruction'][i]}\n\n### 回答:\n{example['output'][i]}<|im_end|>"
        output_texts.append(text)
    return output_texts

In [10]:
args = SFTConfig(
    output_dir='./output',
    num_train_epochs=2,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=8,
    save_strategy="no",
    logging_steps=20,
    lr_scheduler_type="constant",
    save_total_limit=1,
    fp16=True,
    neftune_noise_alpha=5,
    auto_find_batch_size=True,
)

In [11]:
trainer = SFTTrainer(
    model,
    args=args,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
    peft_config=peft_config,
)

In [12]:
trainer.train()
trainer.save_model()

Step,Training Loss
20,3.361700
40,2.812900
60,2.581800
80,2.428600
100,2.331000
120,2.360300
140,2.367300
160,2.340000
180,2.294300
200,2.286500


In [13]:
def generate(model, tokenizer, input_text):
    input_text = f"### 質問:\n{input_text}\n\n### 回答:"
    inputs = tokenizer(input_text, return_tensors='pt').to(model.device)
    
    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            max_length = 512,
            do_sample = True,
            temperature = 0.8,
            top_p = 0.9,
            top_k = 0,
            repetition_penalty = 1.1,
            num_beams = 1
        )
    
    generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    return generated_text

In [22]:
input_text="AIとはなんですか？"
generated_text = generate(model, tokenizer, input_text)
print(generated_text)

### 質問:
AIとはなんですか？

### 回答:：
我、りんえもんは思う。 AI（人工知能）は、データを処理し、機械学習を行って、特定のタスクに必要な情報を生産したり学習させたりすることができますでござる。

AIは、人間のような思考や行動能力と比較して、より多くの効率的な方法を導入できますでござる。例えば、強力な自動車の設計者によって、その自動車が何回乗車してもすぐに運転できますでござる。

このように、AIは、訓練された従業員のように効率的に働くことができるようになりますでござる。コンピュータやスマートフォンなどのデバイスを使用して、モデルを学習することで、より効果的である領域を学習することができるようになりますでござる。

しかし、最近では、AIは非常に複雑な問題に対して対応でき、新しいことに対応することは困難になっていますでござる。また、オーバーアシスタンスでもあり、誤解やリスクのある研究結果を示すなど、様々な問題に対する解決策を考えたりする必要があることもありませぬ。知らんけど。
